## Preparing queries for Abstract Sifter

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'

In [3]:
df = pd.read_excel(interim_dir+'secondary_categorisations_new_universe_061123.xlsx', index_col = [0])

In [4]:
df = df[['dtxsid','Substance_Name', 'Substance_CASRN']].dropna()

In [5]:
df['query'] = df['Substance_Name'] + ' OR ' + df['Substance_CASRN']+" [tiab]"

In [6]:
df['query'][0]

'N-Ethyl-N-((pentadecafluoroheptyl)sulphonyl)glycine OR 68957-63-1 [tiab]'

In [7]:
df['Column3'] = df.apply(lambda row: row['Substance_Name'] + ' OR '+ row['Substance_CASRN'] if ' ' not in row['Substance_Name'] else f'{row["Substance_Name"]} OR {row["Substance_CASRN"]}', axis=1)

In [8]:
df['term1'] = [e.split("OR")[0] for e in df['Column3']]
df['term2'] = [e.split("OR")[1] for e in df['Column3']]

In [9]:
df['term3'] = [f"{e}" for e in df['term1']]

In [10]:
df['term1'] = [e.strip() for e in df['term1']]

In [11]:
df['term1'][0]

'N-Ethyl-N-((pentadecafluoroheptyl)sulphonyl)glycine'

In [12]:
df['term3'] = ['"' + s + '"' for s in df['term1']]

In [13]:
df['term4']= ['"' + s + '"' for s in df['term2']]

In [14]:
import time
import requests
import json

In [15]:
my_lst = []
#url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=1&sort=relevance&term=({term2} OR {term3})"
for i, row in df.iterrows():
    term, term1, term2,  = row['dtxsid'], row['term3'], row['term4']
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=1&sort=relevance&term=({term1} OR {term2})[tiab]"
    #print(url)
    try:
        response = requests.get(url).json()
        a = response['esearchresult']['count']
        time.sleep(1)
    except Exception as error:
        print(error)
    finally:
        my_lst.append([term,a])

'count'
'count'
'count'
'count'
'count'
'count'
'esearchresult'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'esearchresult'
'esearchresult'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
'count'
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


In [19]:
pd.DataFrame(my_lst, columns = ['dtxsid', 'article_count']).to_csv(interim_dir+'pubmed_counts_081123.csv')

In [22]:
pd.DataFrame(my_lst, columns = ['dtxsid', 'article_count'])['article_counts'].astype(float).hist()

KeyError: 'article_counts'